# Retrieval-Augmented Generation (RAG) Model for QA Bot on P&L Data

### Problem Statement:
Develop a Retrieval-Augmented Generation (RAG) model for a Question Answering (QA) bot that can
process financial terms and insights from a Profit & Loss (P&L) table extracted from PDF documents.

### Task Requirements:

- Implement a RAG-based model to handle questions related to a P&L table extracted fromPDF documents.
- Use a vector database (such as Pinecone) to store and retrieve document embeddings of financial terms and data points efficiently.
- Parse P&L data from PDF documents into a structured format, such as tables or key-value pairs, before storing embeddings.
- Test the model with several financial queries and show how accurately it retrieves and generates responses from the dataset.

## CONTENTS
- Imports and setting device
- Parsing PDF file and preprocessing
- Emdedding genertion and Vectorbase creation
- RAG pipeline
- Retrievel 
- Augment and generate
- Testing with queries

## IMPORTS

In [1]:
import torch
from torch import autocast
import pandas as pd
import pdfplumber
import camelot
import chromadb
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np

/Users/gokulgopank/Documents/Sample_set/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setting device

GPU if available is enabled for faster training and inference.

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


## Parsing PDF file and preprocessing

First need to extract the relevant pages that could have the profit loss table. The PL table usually contains the words “Statement of Profit and Loss" , "Revenue" and "Expenses”. This logic is used to extract the relevant pages. Another point is that the “Contents” page which appear in beginning of the document will also be having these terms. Hence the page with PL table will be the second one having all the said words. This logic is used to extract the correct page with PL table.

In [22]:
def extract_page_no(pdf_path):
    relevant_pages=[]
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text()
            # Check if "Profit and Loss,Renenue and Expenses." is in the page text
            if "Statement of Profit and Loss" in text and "Revenue" in text and "Expenses" in text:
                # Print the page number
                print(f"Found relevant content on page {i}")
                relevant_pages.append(i)
        return str(relevant_pages[1])

# Example usage
pdf_path = 'Sample Financial Statement.pdf'
x = extract_page_no(pdf_path)
print(x)

Found relevant content on page 1
Found relevant content on page 3
3


The table is extracted using camelot library. It is then preprocessed to removed unwanted rows and column. The columns are also edited to make a coherent structure.

In [4]:

# PDF Extraction Function (adapt as needed for your specific PDF structure)
def extract_profit_loss_tables(pdf_path,page):
    tables = camelot.read_pdf(pdf_path, pages=page, flavor='stream')
    if tables:
        df = tables[0].df
        df = df.iloc[2:]
        df = df.reset_index(drop=True)
        df.columns = df.iloc[0]
        df = df.drop(index=[0, 1])
        df[['Year ended March 31,2024', 'Year ended March 31,2023']] = df['Year ended March 31,'].str.split('\n', expand=True)
        df = df.drop(columns=['Note No.', 'Year ended March 31,'])
        df.columns.values[1] = 'Three months ended March 31,2024'
        df.columns.values[2] = 'Three months ended March 31,2023'
        df = df.reset_index(drop=True)
        return df
    else:
        print("No tables found on the specified page.")
        return pd.DataFrame() #Return empty dataframe to avoid errors
    
pdf_path="Sample Financial Statement.pdf"
pl_page = extract_page_no(pdf_path)
pl_table=extract_profit_loss_tables(pdf_path,pl_page)
pl_table

Found relevant content on page 1
Found relevant content on page 3


,Condensed Consolidated Statement of Profit and Loss for the,"Three months ended March 31,2024","Three months ended March 31,2023","Year ended March 31,2024","Year ended March 31,2023"
0,Revenue from operations,"37,923","37,441","153,670","146,767"
1,"Other income, net","2,729",671,"4,711","2,701"
2,Total income,"40,652","38,112","158,381","149,468"
3,Expenses,,,,None
4,Employee benefit expenses,"20,393","20,311","82,620","78,359"
5,Cost of technical sub-contractors,"2,967","3,116","12,232","14,062"
6,Travel expenses,471,426,"1,759","1,525"
7,Cost of software packages and others,"3,687","2,886","13,515","10,902"
8,Communication expenses,147,171,677,713
9,Consultancy and professional charges,489,387,"1,726","1,684"


## Emdedding genertion and Vectorbase creation

Different open source vector base approaches were tried such as FAISS and Chromadb.
ChromaDB is selected as it is designed for AI and Retrieval-Augmented Generation (RAG)applications.
It is optimized for fast similarity search and can store, index, and retrieve embeddings efficiently.

ChromaDB client is first initialised and create a collection named "financial_data in the persistent" database.
Collections in ChromaDB are used to store vector data along with metadata.  This is where you would store vectors (e.g., embeddings of financial documents) and associated metadata (e.g., document IDs, timestamps, etc.).
ChromaDB is a vector database designed to manage and store vector embeddings. It allows for efficient similarity search and retrieval of high-dimensional vectors, which is especially useful for applications like semantic search, machine learning, and natural language processing tasks. ChromaDB can handle storing, querying, and indexing embeddings, making it a useful tool for building systems that rely on vector search and retrieval.

In [5]:
# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="financial_data")

The 'all-MiniLM-L6-v2' is used as pretrained embedding model from the SentenceTransformers library. It is a lightweight model based on Microsoft's MiniLM architecture, which balances speed and accuracy. Is is used to generate fixed-length vector dense representation  called embeddings from text.
Each row is converted into a high-dimensional vector representation. These embeddings are stored as NumPy arrays. Finally the model is moved to the selected device for faster processing.


In [6]:
# Embedding and FAISS Setup
embedding_model_name = 'all-MiniLM-L6-v2'
embedding_model= SentenceTransformer(embedding_model_name)
embedding_model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

Next need to extract embeddings from tabular data andstore them in ChromaDB for efficient retrieval.

First converts all DataFrame values to strings and aggregateeach row into a single space-separated string andconvert the rows into a list of strings .
Use embedding model to generate vector embeddings for each row's text and converts the embeddings from a tensor to a NumPy array.
Store each embedding in ChromaDB  using the row index as the unique identifier. Also store the original text as metadata.

In [7]:
# Function to store embeddings in ChromaDB
def embed_and_store(df):
    if df.empty:
        return
    
    rows_text = df.astype(str).agg(' '.join, axis=1).tolist()
    embeddings = embedding_model.encode(rows_text, convert_to_tensor=True, device=device).cpu().numpy()
    for i, embedding in enumerate(embeddings):
        collection.update(
            ids=[str(i)],
            embeddings=[embedding.tolist()],
            metadatas=[{"row": rows_text[i]}]
        )


In [8]:
stored_data = collection.get(include=[ 'embeddings','metadatas'])
print(stored_data)

{'ids': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32'], 'embeddings': array([[ 0.01805673, -0.05449116, -0.07919173, ..., -0.07793081,
         0.04185454, -0.06581722],
       [ 0.03873873, -0.05528183, -0.01569921, ..., -0.04594269,
        -0.02963915, -0.04241281],
       [ 0.06413926, -0.03403589,  0.00919265, ..., -0.06510878,
        -0.01722785, -0.08938234],
       ...,
       [-0.04281837, -0.04527131,  0.00536274, ..., -0.08641743,
        -0.02762058, -0.05120561],
       [-0.00147541,  0.08183503, -0.12176751, ..., -0.11915076,
         0.01085957,  0.0036928 ],
       [-0.04783782, -0.04752753, -0.05690166, ..., -0.0609794 ,
         0.0648248 , -0.00879265]]), 'documents': None, 'uris': None, 'data': None, 'metadatas': [{'row': 'Revenue from operations 37,923 37,441 153,670                            146,767'}, {'row': 'Other incom

So embeddings have been sucessfully stored in chromadb database.

## RAG pipeline

### Retrievel 
Next need to  use ChromaDB as a vector search index to retrieve the most relevant rows from a DataFrame based on the similarity between a query and the stored vector representations of data. Query is he input string for which you want to find relevant rows.

Next need to retrieves the top k relevant rows from a DataFrame based on a similarity search using embeddings. For this encode the query, performs a vector search, extract matching rows, and filter the DataFrame to return only the relevant rows.

First the query input is transformed into a query embedding using embedding model. The model takes the query string and encodes it into a tensor representation and then converted to a NumPy array.
Next performs a query on a vector database giving top k closest matches in terms of similarity to the query embedding.
Then get the filtered DataFrame, which contains only the rows whose combined text matches the retrieved rows from the vector search.


In [9]:
# Retrieval Function
def retrieve_relevant_rows(query, df, top_k=7):
    if df.empty:
        return pd.DataFrame()
    query_embedding = embedding_model.encode(query, convert_to_tensor=True, device=device).cpu().numpy().tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    retrieved_texts = [res["row"] for res in results["metadatas"][0]]
    return df[df.astype(str).agg(' '.join, axis=1).isin(retrieved_texts)]

### Augment and generate

Next need to select the generation model and prepare context according to its structure.
Initially the TAPAS model, which stands for Tabular Pretrained Language Model, which a deep learning model designed for handling tabular data (data presented in tables, often seen in spreadsheets or databases). Unlike traditional language models like BERT or GPT, which are trained on textual data, TAPAS is specifically designed to interpret and answer questions about data presented in table form. But deepseek models are better open-source alternative and higher efficiency.

In [10]:
# LLM Inference
llm_model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModelForCausalLM.from_pretrained(llm_model_name, torch_dtype=torch.float16).to(device)

Next need to convert  content into a formatted string representation which can be input to the question answering llm model ie deepseek model.

For this, first convert each row into a string representation where:column names (col) and corresponding values (value) are formatted as key-value pairs are then joined using a comma (, ) separator.The formatted row string is prefixed with "Row {index}:", indicating its original position in the DataFrame.
The result is appended to the final list. Finally, all formatted rows are joined together using newline characters (\n) to create a structured multi-line string.


In [11]:
# Context Preparation
def prepare_context(relevant_rows):
    if relevant_rows.empty:
        return ""
    formatted_data = []
    for index, row in relevant_rows.iterrows():
        row_str = ", ".join(f"{col}: {value}" for col, value in row.items())
        formatted_data.append(f"Row {index}: {row_str}")
    
    return "\n".join(formatted_data)

In [12]:
def answer_question(question, context):
    if not context:
        return "No relevant context found."
    prompt = f"Context:\n{context}\n\nQuestion:\n{question}\n\nAnswer:"
    inputs = llm_tokenizer(prompt, return_tensors="pt").to(device)
    inputs['input_ids'] = inputs['input_ids'].to(dtype=torch.long)
    with torch.no_grad():
        with autocast(str(device)):
            outputs = llm_model.generate(
                **inputs, max_new_tokens=200, temperature=0.01, top_p=0.9, do_sample=True,
                pad_token_id=llm_tokenizer.eos_token_id
            )
    return llm_tokenizer.decode(outputs[0], skip_special_tokens=True).split("Answer:")[-1].strip()

Next need to take a question and a context as input and generates an answer based on the given context using the deepsake LLM. 
The steps are as follows:
-Creating the Prompt:
Construct a prompt string that follows a structured format:"Context:\n{context}\n\nQuestion:\n{question}\n\nAnswer:"This format helps the LLM understand that it needs to answer the question based on the given context.
-Tokenizing the Input:
The llm tokenizer converts the prompt into tokenized input tensors Also the input tensor  is explicitly cast to torch.long to ensure compatibility with the model.
-Generating the Answer:
autocast(device) is used for mixed precision to improve performance on GPUs. Generate text with the by setting parameters such as max no of tokens, padding and temperature.
-Decoding the Output:
The generated output tokens are converted back into text using llm tokenizer. The n remove unnecessary text before "Answer:" and trimming extra spaces.

Now create the RAG pipeline. We will get the relevant rows and answer to the query.

In [13]:
def retrieve_and_generate_answer(query, df):
    relevant_rows = retrieve_relevant_rows(query, df)
    context = prepare_context(relevant_rows)
    return relevant_rows, answer_question(query, context)

In [23]:
# function to process the uploaded PDF and query
def process_pdf_and_query(pdf_file, query):
    pl_page=extract_page_no(pdf_path)
    # Extract tables from the uploaded PDF
    pnl_table = extract_profit_loss_tables(pdf_file,pl_page)
    embed_and_store(pnl_table)
    relevant_rows,answer = retrieve_and_generate_answer(query, pnl_table)
    return relevant_rows, answer

### Testing with queries

Now lets test the model with some financialqueries.

In [24]:
pdf_file="Sample Financial Statement.pdf"
query="What is the gross profit for Q3 2024?"

relevant_rows, answer = process_pdf_and_query(pdf_file, query)
print("Answer:", answer)

Found relevant content on page 1
Found relevant content on page 3
3
Answer: The gross profit for Q3 2024 is 7,975.


In [ ]:
print(relevant_rows)

In [ ]:
query="How do the net income and operating expenses compare for Q1 2024?"

relevant_rows, answer = process_pdf_and_query(pdf_file, query)
print("Answer:", answer)


In [ ]:
print(relevant_rows)

Tasks completed:

- Implemented a RAG-based model to handle questions related to a P&L table extracted fromPDF documents.
- Used chromadb vectorbase to store and retrieve document embeddings of financial terms and data points efficiently.
- Parsed P&L data from PDF documents into a structured format, such as tables or key-valuepairs, before storing embeddings.
- Teste the model with several financial queries and show how accurately it retrieves and
generates responses from the dataset.